In [2]:
!pip install flask-cors opencv-python matplotlib Flask pyngrok numpy requests pybase64 google-generativeai


In [3]:
import cv2
import matplotlib.pyplot as plt
from flask import *
from pyngrok import ngrok
import json
import io
import base64
import numpy as np
import requests


In [4]:
port_no = 3030
from flask_cors import CORS

In [5]:
def highlight_regions(image_path, predictions):

    image = cv2.imread(image_path)
    for pred in predictions:
        x, y, width, height = int(pred['x']), int(pred['y']), int(pred['width']), int(pred['height'])

        half_width = width // 2
        half_height = height // 2
        x1 = x - half_width
        y1 = y - half_height
        x2 = x + half_width
        y2 = y + half_height

        cv2.rectangle(image, (x1, y1), (x2, y2), ( 0,255,0), 2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(image,  f"{pred['class']} ({pred['confidence']:.2%})", (x1, y1 - 10), font, 0.75, (  0,255,0), 1, cv2.LINE_AA)

    return image


In [10]:

def generate_analysis(disease_name, predictions , base64String):
    if isinstance(predictions, list):
        predictions_str = json.dumps(predictions)
    elif isinstance(predictions, dict):
        # Formatting predictions to ensure they are not outputted as decimals
        formatted_predictions = {key: int(value) if isinstance(value, float) and value.is_integer() else value for key, value in predictions.items()}
        predictions_str = json.dumps(formatted_predictions, indent=4)
    else:
        raise ValueError("predictions must be either a dictionary or a list")

    # Creating the prompt
    prompt = """
You are an assistant and you are given the output of the radiology image analysis machine learning algorithm.
You are supposed to elaborate on the output and provide further symptoms that can be used for further analysis.
Remember to use proper grammar.

The following is the output of the analysis machine learning algorithm of %s and the region of interest:

"predictions": %s

Write in 75 words and only important information like symptoms and precautions in a form in which the doctor can understand. If you can't tell everything, elaborate prediction itself.

""" % (disease_name, predictions_str)
    

    url = "http://localhost:11434/api/generate"
    
    # Prepare payload
    payload = {
      "model": "mistral",
      "prompt": prompt,
      "stream": False,
    }
    response = requests.post(url, data=json.dumps(payload))
    response_data = response.json()

    return response_data["response"]
    
